# Libaries

In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.autograd import Variable

import os

import matplotlib.pyplot as plt

# Generall

In [76]:
print('Cuda is available:',torch.cuda.is_available(),'\n')
      
      
!nvidia-smi

Cuda is available: False 

Fri Jun  2 11:33:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.77       Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |    475MiB /  2048MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                             

# Hyperparameters

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


batch_size = 16


num_epochs = 5
learn_rate = 0.0004

# Load Data

In [78]:
# create folder structure if it does not exist
folder_path = "../../data/"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


#MNIST Dataset
    #Create Folder, transorm to Tensor
train_dataset = torchvision.datasets.MNIST(
    root=folder_path, train=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.01307),(0.3081))]), download=True
)

test_dataset = torchvision.datasets.MNIST(
    root=folder_path, train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.01307),(0.3081))])
)

#Data Loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

# Model

In [79]:
class MNIST_De_Class(nn.Module):
    def __init__(self):
        super(MNIST_De_Class, self).__init__()
        # self.input = nn.Linear(28 * 28, 20)  # input
        # self.hidden1 = nn.Linear(20, 20)  # hidden1
        # self.hidden2 = nn.Linear(20, 20)  # hidden2
        # self.out = nn.Linear(20, 10)  # output
      
        self.network = nn.Sequential(
            nn.Linear(28 * 28, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # x = self.input(x)
        # x = nn.ReLU(x)
        # x = self.hidden1(x)
        # x = nn.ReLU(x)
        # x = self.hidden2(x)
        # x = nn.ReLU(x)
        # x = self.out(x)
        # x = nn.Softmax(x)
        # print(x.size())
        return (self.network(x))

# Train

In [80]:
model1 = MNIST_De_Class().to(device)

#Load Model / Continue Training
continue_training = True


FILE = "model1.pth"

if continue_training:
    model1 = torch.load(FILE).to(device)

#criterion / lf
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.Adam(params=model1.parameters(),lr=learn_rate)

for epoch in range(num_epochs):
    for batch_id, (data,target) in enumerate(train_loader):
        data = data.reshape(-1,28*28).to(device)  #[100, 1,28,28] > [100,734] ?
        target = target.to(device)
        
        output = model1(data) #was macht out?
        loss = criterion(output,target) #hier out?!

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        

    print('Epoche: ',epoch+1, 'Loss: %.5f ' % loss) 

Epoche:  1 Loss: 1.52365 
Epoche:  2 Loss: 1.46970 
Epoche:  3 Loss: 1.58615 
Epoche:  4 Loss: 1.57624 
Epoche:  5 Loss: 1.49536 


# Save Model

In [81]:
torch.save(model1,FILE)